In [ ]:
FATIH RAKA KSATRIA

215150300111011

COMPUTER VISION - B

In [ ]:
!unzip Images.zip

In [ ]:
!unzip "C:\Users\fatih\Downloads\Airplanes_Annotations.zip"

In [ ]:
pip install tensorflow


In [8]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [9]:
path = "stop_sign"
annot = "stop_sign_annotations_converted"

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

# Path folder gambar dan file anotasi
path = "stop_sign"
annotation_file = "stop_sign_annotations_converted.txt"

# Membaca file anotasi
df = pd.read_csv(annotation_file, sep=",", header=None, names=["annotation_file", "bbox"])

# Memisahkan bounding box menjadi kolom terpisah
df[['x1', 'y1', 'x2', 'y2']] = df['bbox'].str.split(expand=True).astype(int)

# Loop melalui setiap gambar dan anotasinya
for index, row in df.iterrows():
    # Membentuk nama file gambar berdasarkan nomor pada nama file anotasi
    image_number = row['annotation_file'].split("_")[1].split(".")[0]
    filename = f"image_{image_number.zfill(4)}.jpg"  # Menghasilkan format image_0001.jpg, image_0002.jpg, dst.
    
    # Membaca gambar
    img = cv2.imread(os.path.join(path, filename))
    
    # Cek apakah gambar berhasil dibaca
    if img is not None:
        # Plot gambar
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
        # Menambahkan bounding box yang terkait dengan gambar ini
        x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        # Tampilkan gambar dengan bounding box
        plt.figure()
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(filename)  # Tambahkan judul gambar untuk referensi
        plt.axis('off')  # Matikan sumbu untuk tampilan yang lebih baik
        plt.show()
    else:
        print(f"Could not read image {filename}. Check if the file exists and path is correct.")


In [13]:
cv2.setUseOptimized(True);
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [ ]:
pip install --user opencv-contrib-python


In [ ]:
im = cv2.imread(os.path.join(path,"image_0025.jpg"))
ss.setBaseImage(im)
ss.switchToSelectiveSearchFast()
rects = ss.process()
imOut = im.copy()
for i, rect in (enumerate(rects)):
    x, y, w, h = rect
#     print(x,y,w,h)
#     imOut = imOut[x:x+w,y:y+h]
    cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
# plt.figure()
plt.imshow(imOut)

In [18]:
train_images=[]
train_labels=[]

In [20]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [22]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [ ]:
for e, row in df.iterrows():
    annotation_filename = row['annotation_file']
    # Dapatkan nama gambar yang sesuai
    image_number = annotation_filename.split("_")[1].split(".")[0]
    filename = f"image_{image_number.zfill(4)}.jpg"

    image = cv2.imread(os.path.join(path, filename))

    if image is None:
        print(f"Gambar {filename} tidak dapat dibaca. Periksa format atau path.")
        continue

    gtvalues = [{"x1": row['x1'], "y1": row['y1'], "x2": row['x2'], "y2": row['y2']}]

    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = image.copy()
    counter = 0
    falsecounter = 0
    flag = 0
    
    for e, result in enumerate(ssresults):
        if e < 2000 and flag == 0:
            for gtval in gtvalues:
                x, y, w, h = result
                iou = get_iou(gtval, {"x1": x, "x2": x+w, "y1": y, "y2": y+h})
                print(f"IOU: {iou}, Counter: {counter}, False Counter: {falsecounter}")
                if counter < 30:
                    if iou > 0.70:
                        timage = imout[y:y+h, x:x+w]
                        resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                        train_images.append(resized)
                        train_labels.append(1)
                        counter += 1
                if falsecounter < 30:
                    if iou < 0.3:
                        timage = imout[y:y+h, x:x+w]
                        resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                        train_images.append(resized)
                        train_labels.append(0)
                        falsecounter += 1

                # Menghentikan pencarian jika batas tercapai
                if counter >= 30 and falsecounter >= 30:
                    print("Menghentikan pencarian karena batas tercapai.")
                    flag = 1  # Menghentikan pencarian



In [25]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

In [ ]:
X_new.shape

In [ ]:
pip install --upgrade tensorflow keras


In [30]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [32]:
from tensorflow.keras.applications.vgg16 import VGG16

# Cek jika model sudah ada di local
try:
    vggmodel = VGG16(weights='imagenet', include_top=True)
except Exception as e:
    print(f"Error loading model: {e}")

In [ ]:
vggmodel.summary()

In [ ]:
for layers in (vggmodel.layers)[:15]:
    print(layers)
    layers.trainable = False

In [50]:
X= vggmodel.layers[-2].output

In [52]:
predictions = Dense(2, activation="softmax")(X)

In [53]:
model_final = Model(inputs=vggmodel.input, outputs=predictions)


In [54]:
from keras.optimizers import Adam

# Menggunakan argumen learning_rate
opt = Adam(learning_rate=0.0001)


In [55]:
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])

In [ ]:
model_final.summary()

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [62]:
class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)

In [63]:
lenc = MyLabelBinarizer()
Y =  lenc.fit_transform(y_new)

In [64]:
X_train, X_test , y_train, y_test = train_test_split(X_new,Y,test_size=0.10)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [72]:
trdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
traindata = trdata.flow(x=X_train, y=y_train)
tsdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
testdata = tsdata.flow(x=X_test, y=y_test)

In [96]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [74]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Menggunakan save_freq sebagai pengganti period
checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.keras", monitor='val_loss', verbose=1, 
                             save_best_only=True, save_weights_only=False, 
                             mode='auto', save_freq='epoch')

# Menginisialisasi EarlyStopping
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')


In [ ]:
# Memastikan bahwa generator `traindata` dan `testdata` sudah kompatibel
hist = model_final.fit(
    traindata,
    steps_per_epoch=10,
    epochs=100,
    validation_data=testdata,
    validation_steps=2,
    callbacks=[checkpoint, early],
    verbose=1  # Memastikan tampilan log training
)

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(hist.history["acc"])
# plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss","Validation Loss"])
plt.show()
plt.savefig('chart loss.png')

In [ ]:
im = X_test[100]
plt.imshow(im)
img = np.expand_dims(im, axis=0)
out= model_final.predict(img)
if out[0][0] > out[0][1]:
    print("sign")
else:
    print("not sign")

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Inisialisasi selective search
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

z = 0
path = 'stop_sign'  # Ganti dengan path ke folder gambar Anda
for e, i in enumerate(os.listdir(path)):
    if i.startswith("image_006"):  # Misalnya memproses gambar yang namanya diawali '4'
        z += 1
        img = cv2.imread(os.path.join(path, i))
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = img.copy()
        
        for e, result in enumerate(ssresults):
            if e < 20:
                x, y, w, h = result
                # Print for debugging
                print(f"Processing proposal {e + 1}: x={x}, y={y}, w={w}, h={h}")
                
                timage = imout[y:y + h, x:x + w]
                resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)
                
                out = model_final.predict(img)
                print(f"Prediction output for proposal {e + 1}: {out}")  # Output prediksi untuk setiap proposal
                
                if out[0][0] > 0.65:
                    cv2.rectangle(imout, (x, y), (x + w, y + h), (0, 255, 0), 1, cv2.LINE_AA)

        # Menampilkan gambar hasil dengan bounding box
        plt.figure()  # Membuat figure baru untuk setiap gambar
        plt.imshow(cv2.cvtColor(imout, cv2.COLOR_BGR2RGB))  # Konversi BGR ke RGB
        plt.title(f"Image {i} with Predictions")  # Menambahkan judul
        plt.axis('off')  # Menyembunyikan axis
        plt.show()  # Menampilkan gambar
